In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity


2023-08-15 01:12:50.278410: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 01:12:50.628711: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 01:12:50.629774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 01:12:52.723116: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('Dataset.csv', usecols=['StockCode', 'Description'])
df = df.dropna()
df = df.drop_duplicates()

In [3]:
# print n of unique StockCode
print("Number of unique StockCode: ", len(df['StockCode'].unique()))

# get duplicate StockCode
duplicateStockCode = df[df.duplicated(['StockCode'])]
print("Number of duplicate StockCode: ", len(duplicateStockCode))
# print the first 5 duplicate StockCode
print(duplicateStockCode.head())
# print all of StockCode 22632
print("All of StockCode 22632: ")
print(df[df['StockCode'] == '22778'])

# decide not to remove duplicate StockCode because they may have different description

Number of unique StockCode:  3958
Number of duplicate StockCode:  834
     StockCode                         Description
257      22632           HAND WARMER RED RETROSPOT
2343     22972              CHILDRENS SPACEBOY MUG
3305     22965  3 TRADITIONAl BISCUIT CUTTERS  SET
3414     22776        CAKESTAND, 3 TIER, LOVEHEART
3415     22778                GLASS BELL JAR SMALL
All of StockCode 22632: 
       StockCode           Description
138        22778    GLASS CLOCHE SMALL
3415       22778  GLASS BELL JAR SMALL
478932     22778                 check


In [4]:
item_tuples = [(row['StockCode'], row['Description']) for index, row in df.iterrows()]

# print the first 10 tuples
print(item_tuples[:10])

[('85123A', 'WHITE HANGING HEART T-LIGHT HOLDER'), ('71053', 'WHITE METAL LANTERN'), ('84406B', 'CREAM CUPID HEARTS COAT HANGER'), ('84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE'), ('84029E', 'RED WOOLLY HOTTIE WHITE HEART.'), ('22752', 'SET 7 BABUSHKA NESTING BOXES'), ('21730', 'GLASS STAR FROSTED T-LIGHT HOLDER'), ('22633', 'HAND WARMER UNION JACK'), ('22632', 'HAND WARMER RED POLKA DOT'), ('84879', 'ASSORTED COLOUR BIRD ORNAMENT')]


In [5]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
def embed(input):
  return model(input)


def calculate_cosine_similarity(vec_a, vec_b):
    vec_a = np.array(vec_a).reshape(1, -1)
    vec_b = np.array(vec_b).reshape(1, -1)
    
    similarity = cosine_similarity(vec_a, vec_b)
    return similarity[0][0]

2023-08-15 01:12:57.299662: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 01:12:57.301506: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
test_item = '22778'

# want the longest description for the item from the df
all_description = df[df['StockCode'] == test_item]['Description']
# det the longest str in all_description
longest_description = max(all_description, key=len)
print(longest_description)

description_embedding = embed([longest_description])

# get the 10 most similar items from item_tuples using cosine similarity
similar_items = []
for item in item_tuples:
    if(item[0] == test_item):
        continue
    item_embedding = embed([item[1]])
    similarity = cosine_similarity(description_embedding, item_embedding)
    # if similairty is higher than all similarities in similar_items, add it to similar_items 
    if len(similar_items) < 10 and similarity > 0.4:
        similar_items.append((item[0], item[1], similarity))
    else:
        for i in range(len(similar_items)):
            if similarity > similar_items[i][2]:
                similar_items[i] = (item[0], item[1], similarity)
                break

# print the 10 most similar items
for item in similar_items:
    print(item)

GLASS BELL JAR SMALL
('22777', 'GLASS BELL JAR LARGE', array([[0.9253808]], dtype=float32))
('85146', 'JARDIN ETCHED GLASS SMALL BELL JAR', array([[0.8178096]], dtype=float32))
('85145', 'JARDIN ETCHED GLASS LARGE BELL JAR', array([[0.7442939]], dtype=float32))
('23089', 'GLASS BON BON JAR', array([[0.6156906]], dtype=float32))
('23021', 'GLASS BONNE JAM JAR', array([[0.6015152]], dtype=float32))
('23016', 'GLASS TWIST BON BON JAR', array([[0.5970974]], dtype=float32))
('23020', 'GLASS SONGBIRD STORAGE JAR', array([[0.5664035]], dtype=float32))
('23014', 'GLASS APOTHECARY BOTTLE ELIXIR', array([[0.51625276]], dtype=float32))
('84750A', 'PINK SMALL GLASS CAKE STAND', array([[0.50708103]], dtype=float32))
('23012', 'GLASS APOTHECARY BOTTLE PERFUME', array([[0.49571562]], dtype=float32))
